# Matrix multiplication

In [41]:
import torch
from torch import tensor


def matmul(a, b):
    ar, ac = a.shape  # n_rows * n_cols
    br, bc = b.shape
    assert ac == br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            for k in range(ac):
                c[i, j] += a[i, k] * b[k, j]
    return c


Say we are working with 5 MNIST images each of size (28 by 28), we turn them into 10 activations.

In [42]:
m1 = torch.randn(5, 28 * 28)
m2 = torch.randn(28 * 28, 10)
matmul(m1, m2)


tensor([[ 58.6165, -10.1676,  21.6245,  -2.6803, -64.2527, -28.5411, -10.5376,
          36.7678,  -4.2712,   8.9385],
        [ 26.7987,   7.4983, -40.0363, -14.0619,  11.2322, -11.8173,  20.1772,
          -8.1895,  15.2847,   7.0598],
        [  0.8149,   9.0984,  27.9498, -15.3219, -13.3059,   7.1128, -18.9964,
         -29.5449, -29.8525,   6.2420],
        [-30.5308,   9.2034,  14.0495, -24.3064, -30.7182,  23.1877,  17.8508,
          43.2565, -27.7722,  -5.1529],
        [  3.4262,   0.3862, -39.1668, -54.1615,   3.0668, -44.7902,  -5.7787,
          18.5482,  30.4846,  -6.3429]])

However, this is slow. We will move this implementation to Pytorch, Basic operation with two tensors are applied elementwise in Pytorch.    

In [43]:
a = tensor([10.0, 6, -4])
b = tensor([2.0, 8, 7])
a + b, a < b

(tensor([12., 14.,  3.]), tensor([False,  True,  True]))

We will replace the inner loop with it's PyTorch equivalent:

In [44]:
def matmul(a, b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac == br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            c[i, j] = (a[i] * b[:, j]).sum()
    return c


We will further leverage broadcasting to improve this function.

# Broadcasting
Broadcasting works by comparing corresponding dimensions of two vectors/scalars.



We can broadcast a scalar to a vector/matrix:

In [45]:
a = tensor([10.0, 6, -4])
a > 0

tensor([ True,  True, False])

In [46]:
m = tensor([[1.0, 2, 3], [4, 5, 6], [7, 8, 9]])
(m - 5) / 2.73

tensor([[-1.4652, -1.0989, -0.7326],
        [-0.3663,  0.0000,  0.3663],
        [ 0.7326,  1.0989,  1.4652]])

We can also broadcast a vector to a matrix

In [47]:
c = tensor([10.0, 20, 30])
m = tensor([[1.0, 2, 3], [4, 5, 6], [7, 8, 9]])
m.shape, c.shape, m + c


(torch.Size([3, 3]),
 torch.Size([3]),
 tensor([[11., 22., 33.],
         [14., 25., 36.],
         [17., 28., 39.]]))

Under the hood, the vector is stretched by calling the `expand_as` method:

In [48]:
c.expand_as(m)

tensor([[10., 20., 30.],
        [10., 20., 30.],
        [10., 20., 30.]])

We can look at how it is stored by calling the `storage` method. Notice that the result of a vector-turned-matrix is not stored as a 3 x 3 (2-d) matrix, but still a 1-d vector.

This is because the vector has a stride of 0, which means when Pytorch goes to the next row by adding the stride, the row doesn't change.

In [49]:
t = c.expand_as(m)
t.storage(), t.stride(), t.shape

( 10.0
  20.0
  30.0
 [torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 3],
 (0, 1),
 torch.Size([3, 3]))

When the # of dimensions are not compatible, Pytorch calls `unsqueeze` method first to add a size 1 dimension. By default, the new dimension is added at the beginning (`unsqueeze(0)`). 

In [50]:
c = tensor([10.0, 20, 30])
m = tensor([[1.0, 2, 3], [4, 5, 6], [7, 8, 9]])
c_1 = c.unsqueeze(1)
c_0 = c.unsqueeze(0)  # what Pytorch does
m.shape, c.shape, c_1.shape, c_0.shape


(torch.Size([3, 3]), torch.Size([3]), torch.Size([3, 1]), torch.Size([1, 3]))

In [51]:
c_0, c_1

(tensor([[10., 20., 30.]]),
 tensor([[10.],
         [20.],
         [30.]]))

In [52]:
c_0 + m, c_1 + m

(tensor([[11., 22., 33.],
         [14., 25., 36.],
         [17., 28., 39.]]),
 tensor([[11., 12., 13.],
         [24., 25., 26.],
         [37., 38., 39.]]))

We can now use broadcasting in `matmul`, which accelerate the function further.

In [53]:
def matmul(a, b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac == br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        # c[i,j] = (a[i,:] * b[:,j]).sum() # previous
        c[i] = (a[i].unsqueeze(-1) * b).sum(dim=0)
    return c


# Forward pass

The forward pass is the process of use input and the model to calculate output (based on matrix multiplication).

In [54]:
def lin(x, w, b):
    return x @ w + b


To define a nn, we put two linear layers together, with a nonlinear function, the activation function, in the middle (since the composition of two linear function is just one linear function, which doesn't amounts to 2 layers).

We initialize the weight for a linear layer:

In [55]:
x = torch.randn(200, 100)
y = torch.randn(200)


w1 = torch.randn(100, 50)
b1 = torch.zeros(50)
w2 = torch.randn(50, 1)
b2 = torch.zeros(1)

A problem with this set of weights is that the output of the model after several layers because very big. That is because after going through several linear transformations, matrix multiplications will lead to a very large number. This can be illustrated by iteratively perform matrix multiplications on a matrix. As we can see, the weight matrix becomes very big quickly. This won't work for deep neural networks.

In [56]:
x = torch.randn(200, 100)
for i in range(50):
    x = x @ torch.randn(100, 100)
x[0:5, 0:5]

tensor([[nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]])

Kaiming He et al. introduced the right scaling factor for neural net with relu in their ResNet paper, which is $\sqrt{2/n_{in}}$, where $n_{in}$ is the number of input. So we initialize according to this approach instead:

In [57]:
import math

x = torch.randn(200, 100)
y = torch.randn(200)

w1 = torch.randn(100, 50) * math.sqrt(2 / 100)
b1 = torch.zeros(50)
w2 = torch.randn(50, 1) * math.sqrt(2 / 50)
b2 = torch.zeros(1)

We define an activation function, the nonlinear function, with **relu**, which clamps a tensor at 0:

In [58]:
def relu(x):
    return x.clamp_min(0.0)

Now we can calculate how an input goes thru the first layer:

In [59]:
l1 = lin(x, w1, b1)
l2 = relu(l1)
l2.mean(), l2.std()


(tensor(0.5728), tensor(0.8428))

And we can define our neural network as, which is also our forward pass.

In [60]:
def model(x):
    l1 = lin(x, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

# Backward pass

The backward pass calculates a loss function, which compute a measure of difference between the output of the forward pass and our label (`y` in our case). In our case, we use mean squared error.

However, we have 1 minor problem. Our model should have an output of shape `[200]` (the shape of `y`), but right now it has a trailing 1 dim.

In [61]:
out = model(x)
out.shape

torch.Size([200, 1])

We get rid of the training 1 with `squeeze` method in our loss function:

In [62]:
def mse(output, targ):
    return (output.squeeze(-1) - targ).pow(2).mean()

Now we need to calculate gradients wrt our loss function, the mean root square. To do so, we apply chain rule. To implement the chain rule for all operations we have, we save gradient of every function we have wrt the inputs.

For mean squared error, the gradient wrt to the input (which is the output of the forward pass) is
$$
\frac{d}{d \text{output}}mse(\text{output}, \text{targ}) = 2(\text{output} - \text{targ}) * 1/n
$$
where $n$ is the # of elem in $output$

For relu, the gradient wrt to the input is 1 or 0 depending on the input. Because, the result of relu is not final, we need to apply chain rule, which mean we need to time the result with the gradient of the next step, `out.g`:
$$
g = g_{relu} \cdot g_{out}
$$
where $g_{relu}$ is:
$$
\frac{d}{d \text{input}}relu(\text{input}) = 
\begin{cases}
1 & \text{if } x > 0 \\
0 & \text{if } x \leq 0
\end{cases}
$$

Similarly, for the linear function we have
$$
g = g_{lin} \cdot g_{out}
$$
where $g_{lin}$ is:
$$
\frac{d}{d \text{input}}lin(\text{input}) = w
$$

This gives use:


In [63]:
def mse_grad(inp, targ):
    # grad of loss with respect to output of previous layer
    inp.g = 2.0 * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]


def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp > 0).float() * out.g


def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = inp.t() @ out.g
    b.g = out.g.sum(0)

The backward pass is simply run these gradient one at a time:

In [64]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2

    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

# Refactor to PyTorch

Next, we want to move the current implementation into pytorch. To do that, we first organize the code into classes where each operation has a forward pass and a backward pass.

In [65]:
class Relu:
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.0)
        return self.out

    def backward(self):
        self.inp.g = (self.inp > 0).float() * self.out.g


class Lin:
    def __init__(self, w, b):
        self.w, self.b = w, b

    def __call__(self, inp):
        self.inp = inp
        self.out = inp @ self.w + self.b
        return self.out

    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)


class Mse:
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out

    def backward(self):
        x = (self.inp.squeeze() - self.targ).unsqueeze(-1)
        self.inp.g = 2.0 * x / self.targ.shape[0]

We will also create a model class to tie everything together.

Here there is something really subtle. Because python is pass by reference, the object relation established in the forward pass gets retained in the backward pass. This make it possible to call `self.out.g` without setting it explicitly.

For the forward pass we have:
```
y1  = lin1(x0)      # lin1.out is y1
r1  = relu(y1)      # relu.out is r1; note: r1 is also lin2.inp
yhat = lin2(r1)     # lin2.out is yhat; note: yhat is also loss.inp
L = loss(yhat, targ)
```
In the backward pass:
```
loss.backward() # loss.inp.g is now set → therefore lin2.out.g is set
lin2.backward() # lin2 reads its out.g (set by loss), writes its inp.g (which is relu.out.g)
relu.backward() # relu reads its out.g (set by lin2), writes its inp.g (which is lin1.out.g)
lin1.backward() # lin1 reads its out.g (set by relu), writes its inp.g (which is x0.g)
```

In [66]:
class Model:
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()

    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)

    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()


Now we are in a position to perform forward and backward pass similar to how pytorch does it:

In [69]:
model = Model(w1, b1, w2, b2)
loss = model(x, y)
model.backward()
w1.g.shape

torch.Size([100, 50])

In Pytorch, we rewrite layers of neural net as child class of `nn.Module`, which is the base structure for Pytorch models. It registers trainable parameters and keeps track of gradients. We derive the equivalent gradient of the layer 1 weight `w1` (note that PyTorch weights' shape, `(out_features, in_features)` is the reverse of how we defined it)

In [ ]:
import torch.nn as nn


class Model2(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)
        )
        self.loss = mse

    def forward(self, x, targ):
        return self.loss(self.layers(x).squeeze(), targ)


print(x.shape, y.shape)
model = Model2(100, 50, 1)
loss = model(x, y)
loss.backward()
print(model.layers[0].weight.grad.shape)


torch.Size([200, 100]) torch.Size([200])
torch.Size([50, 100])
